In [1]:
import os
import numpy as np
import nibabel as nib



# Load the files of intensity images, masks and labels:
intensities_path = '/Users/melikapooyan/Documents/elastix-5.0/Scanner1/Intensity_imgs'
labels_path = '/Users/melikapooyan/Documents/elastix-5.0/Scanner1/gt_labels'

def calling_folders(intensities_path, labels_path):
    intensity_files = [f for f in os.listdir(intensities_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
    labels_files = [f for f in os.listdir(labels_path) if f.endswith('.nii') or f.endswith('.nii.gz')]

    return intensity_files, labels_files

intensity_files, labels_files = calling_folders(intensities_path, labels_path)

def load_and_strip(intensity_file_path):
    intensity_image = nib.load(intensity_file_path)
    affined = intensity_image.affine
    headered = intensity_image.header
    intensity_image = intensity_image.get_fdata()

    # Remove singleton dimensions
    intensity_image = np.squeeze(intensity_image)

    return intensity_image, affined, headered


# Create empty lists to store the loaded images
intensity_images = []
mask_images = []

# Loop through the files and load the images
for intensity_file in intensity_files:
    intensity_file_path = os.path.join(intensities_path, intensity_file)
    intensity_image, affined, headered = load_and_strip(intensity_file_path)
    print(intensity_image.shape)  # Add this line to check the shape
    intensity_images.append(intensity_image)


# Convert the list of intensity images and masks to a NumPy array
intensity_images = np.array(intensity_images)

# Calculate the mean of the intensity images and masks
mean_intensities = np.mean(intensity_images, axis=0)

mean_1 = nib.Nifti1Image(mean_intensities, affined, headered)
nib.save(mean_1, '/Users/melikapooyan/Documents/TVTSets/atlas1/mean_1.nii.gz')

# Extract the mask for each tissue from the labeled image, where every mask represents the tissue regions:
def build_atlas(file_path):
    gt_labels = nib.load(file_path).get_fdata()

    csf_gt = (gt_labels == 1).astype(float)
    gm_gt = (gt_labels == 2).astype(float)
    wm_gt = (gt_labels == 3).astype(float)

    return csf_gt, gm_gt, wm_gt

fixed_image_shaope = (256, 128, 256)
csf_accumulated = np.zeros(fixed_image_shaope)  # The same as the shape of the refrence image during registration
gm_accumulated = np.zeros(fixed_image_shaope)
wm_accumulated = np.zeros(fixed_image_shaope)

labels = []
# List files in the folder
for filename in os.listdir(labels_path):
    file_path = os.path.join(labels_path, filename)
    csf_gt, gm_gt, wm_gt = build_atlas(file_path)

    # Accumulate the segmented regions
    csf_accumulated += np.squeeze(csf_gt)
    gm_accumulated += np.squeeze(gm_gt)
    wm_accumulated += np.squeeze(wm_gt)

    labels_image = nib.load(file_path).get_fdata()
    labels.append(labels_image)


# Calculate the mean to find the label probabilities:
total_images = len(os.listdir(labels_path))
prob_csf = csf_accumulated / total_images
prob_gm = gm_accumulated / total_images
prob_wm = wm_accumulated / total_images

p_csf = nib.Nifti1Image(prob_csf, affined, headered)
p_gm = nib.Nifti1Image(prob_gm, affined, headered)
p_wm = nib.Nifti1Image(prob_wm, affined, headered)
nib.save(p_csf, '/Users/melikapooyan/Documents/TVTSets/atlas1/csf_1.nii.gz')
nib.save(p_gm, '/Users/melikapooyan/Documents/TVTSets/atlas1/gm_1.nii.gz')
nib.save(p_wm, '/Users/melikapooyan/Documents/TVTSets/atlas1/wm_1.nii.gz')

(256, 128, 256)
(256, 128, 256)
(256, 128, 256)
(256, 128, 256)
(256, 128, 256)
(256, 128, 256)
